In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from functools import reduce

import numpy as np
from scipy.optimize import minimize

In [2]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))
    for i in range(m):
        term1 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, v[i][j]) @ np.linalg.matrix_power(sigma_minus, w[i][j]) for j in range(n)])
        term2 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, w[i][j]) @ np.linalg.matrix_power(sigma_minus, v[i][j]) for j in range(n)])
        hamiltonian += term1 + term2
    return hamiltonian

def first_nonzero_index(arr, total_bits=3):
    for i, num in enumerate(arr):
        if num != 0:
            binary_repr = format(i, '0' + str(total_bits) + 'b')
            return binary_repr

In [3]:

num = 3
# print("x1+x2-x3=c")
# for i in range(2**num):
#   B = np.zeros(2**num)
#   B[i] = 1
#   C = calculate_hamiltonian(v, w)
#   print(first_nonzero_index(B, num))
#   print(first_nonzero_index(C.dot(B), num))
#   print("====")

In [4]:
def gnrt_Hd():
  Hd = np.zeros((2**3, 2**3)).astype(np.complex128)
  Hd += (add_in_target(3, 0, GateX) @ add_in_target(3, 1, GateX) + add_in_target(3, 0, GateY) @ add_in_target(3, 1, GateY))
  Hd += (add_in_target(3, 1, GateX) @ add_in_target(3, 2, GateX) - add_in_target(3, 1, GateY) @ add_in_target(3, 2, GateY))
  Hd += (add_in_target(3, 2, GateX) @ add_in_target(3, 0, GateX) - add_in_target(3, 2, GateY) @ add_in_target(3, 0, GateY))
  # Hd += add_in_target(3, 0, GateZ)
  # Hd += add_in_target(3, 1, GateZ)
  # Hd += add_in_target(3, 2, GateZ)
  return Hd


In [5]:
binary_string = '   '.join([format(i, '03b') for i in range(8)])
print(binary_string)

000   001   010   011   100   101   110   111


In [6]:
print("x - y - z = 0")
v = np.array([[1, 0, 1], 
              [1, 1, 0]])
w = np.array([[0, 0, 0], 
              [0, 0, 0]])
C = calculate_hamiltonian(v, w)
# 求解本征值和本征态
eigenvalues, eigenvectors = np.linalg.eig(C)
print(np.round(eigenvalues.real, 4))
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

x - y - z = 0
[ 1.4142 -1.4142 -1.4142 -0.      1.4142  0.      0.      0.    ]

Eigenvalues:
1.4142
Eigenvectors:
['0.7071', '0.0000', '0.0000', '0.0000', '0.0000', '0.5000', '0.5000', '0.0000']

Eigenvalues:
-1.4142
Eigenvectors:
['0.7071', '0.0000', '0.0000', '0.0000', '0.0000', '-0.5000', '-0.5000', '0.0000']

Eigenvalues:
-1.4142
Eigenvectors:
['-0.0702', '0.4975', '0.4975', '0.0000', '0.0000', '0.0497', '0.0497', '-0.7036']

Eigenvalues:
-0.0000
Eigenvectors:
['0.0000', '0.7071', '-0.7071', '0.0000', '0.0000', '-0.0000', '-0.0000', '0.0000']

Eigenvalues:
1.4142
Eigenvectors:
['-0.1921', '-0.4812', '-0.4812', '0.0000', '0.0000', '-0.1358', '-0.1358', '-0.6805']

Eigenvalues:
0.0000
Eigenvectors:
['-0.0000', '0.0000', '-0.0000', '0.0000', '0.0000', '0.7071', '-0.7071', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '1.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '1.0000', '0.

In [7]:
print("x + y - z = 0")
v = np.array([[1, 0, 1], 
              [0, 1, 0]])
w = np.array([[0, 0, 0], 
              [1, 0, 0]])
C = calculate_hamiltonian(v, w)
# 求解本征值和本征态
eigenvalues, eigenvectors = np.linalg.eig(C)
print(np.round(eigenvalues.real, 4))
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

x + y - z = 0
[-1.4142  0.      1.4142  1.4142 -1.4142  0.      0.      0.    ]

Eigenvalues:
-1.4142
Eigenvectors:
['0.5000', '0.0000', '0.0000', '0.5000', '0.0000', '-0.7071', '0.0000', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['-0.7071', '0.0000', '0.0000', '0.7071', '0.0000', '-0.0000', '0.0000', '0.0000']

Eigenvalues:
1.4142
Eigenvectors:
['0.5000', '0.0000', '0.0000', '0.5000', '0.0000', '0.7071', '0.0000', '0.0000']

Eigenvalues:
1.4142
Eigenvectors:
['0.0000', '0.0000', '0.7071', '0.0000', '0.5000', '0.0000', '0.0000', '0.5000']

Eigenvalues:
-1.4142
Eigenvectors:
['0.0000', '0.0000', '0.7071', '0.0000', '-0.5000', '0.0000', '0.0000', '-0.5000']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '0.0000', '-0.0000', '0.0000', '0.7071', '0.0000', '0.0000', '-0.7071']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.

In [8]:
print("x + y = 0")
v = np.array([[1, 0], 
              [0, 0]])
w = np.array([[0, 1], 
              [0, 0]])
C = calculate_hamiltonian(v, w)
num = 2
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")
# 求解本征值和本征态
eigenvalues, eigenvectors = np.linalg.eig(C)
print(np.round(eigenvalues.real, 4))
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

x + y = 0
00
00
====
01
01
====
10
01
====
11
11
====
[3. 1. 2. 2.]

Eigenvalues:
3.0000
Eigenvectors:
['0.0000', '0.7071', '0.7071', '0.0000']

Eigenvalues:
1.0000
Eigenvectors:
['0.0000', '0.7071', '-0.7071', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['1.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '1.0000']


In [9]:
print("x - y = 0")
v = np.array([[1, 1], 
              [0, 0]])
w = np.array([[0, 0], 
              [0, 0]])
C = calculate_hamiltonian(v, w)
num = 2
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = calculate_hamiltonian(v, w)
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")
# 求解本征值和本征态
eigenvalues, eigenvectors = np.linalg.eig(C)
print(np.round(eigenvalues.real, 4))
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

x - y = 0
00
00
====
01
01
====
10
10
====
11
00
====
[3. 1. 2. 2.]

Eigenvalues:
3.0000
Eigenvectors:
['0.7071', '0.0000', '0.0000', '0.7071']

Eigenvalues:
1.0000
Eigenvectors:
['-0.7071', '0.0000', '0.0000', '0.7071']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '1.0000', '0.0000', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '0.0000', '1.0000', '0.0000']


In [10]:
v = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])
w = np.array([[0, 1, 0], 
              [0, 0, 1],
              [1, 0, 0]])
C = calculate_hamiltonian(v, w)
# 求解本征值和本征态
eigenvalues, eigenvectors = np.linalg.eig(C)
print(np.round(eigenvalues.real, 4))
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

[-1. -1.  2.  2. -1. -1.  0.  0.]

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.4082', '0.0000', '0.4082', '-0.8165', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.7705', '-0.3853', '-0.1350', '-0.3853', '-0.1350', '0.2700', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '0.4121', '0.4121', '-0.4044', '0.4121', '-0.4044', '-0.4044', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '-0.4212', '-0.4212', '-0.3949', '-0.4212', '-0.3949', '-0.3949', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.2502', '0.3341', '0.5188', '-0.5842', '-0.4603', '-0.0585', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.1569', '0.3647', '-0.5593', '-0.5217', '0.5043', '0.0550', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000']


In [11]:

num = 3
for i in range(2**num):
  B = np.zeros(2**num)
  B[i] = 1
  C = gnrt_Hd()
  print(first_nonzero_index(B, num))
  print(first_nonzero_index(C.dot(B), num))
  print("====")

000
011
====
001
None
====
010
100
====
011
000
====
100
010
====
101
000
====
110
None
====
111
010
====


In [12]:
import numpy as np

Hd = calculate_hamiltonian(v, w)
C = gnrt_Hd()

AB = np.dot(Hd, C)
BA = np.dot(C, Hd)

# 验证是否对易
print("对易" if np.array_equal(AB, BA) else "不对易")

不对易


In [13]:
# x1+x2+x3=c 的 Hd
eigenvalues, eigenvectors = np.linalg.eig(Hd)
print(eigenvalues)
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

[-1. -1.  2.  2. -1. -1.  0.  0.]

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.4082', '0.0000', '0.4082', '-0.8165', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.7705', '-0.3853', '-0.1350', '-0.3853', '-0.1350', '0.2700', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '0.4121', '0.4121', '-0.4044', '0.4121', '-0.4044', '-0.4044', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '-0.4212', '-0.4212', '-0.3949', '-0.4212', '-0.3949', '-0.3949', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.2502', '0.3341', '0.5188', '-0.5842', '-0.4603', '-0.0585', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.1569', '0.3647', '-0.5593', '-0.5217', '0.5043', '0.0550', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000']


In [14]:
# ∑σz x1+x2+x3 的 C
eigenvalues, eigenvectors = np.linalg.eig(C)
print(eigenvalues.real)
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

[-2.  4. -2.  4. -2. -2.  0.  0.]

Eigenvalues:
-2.0000
Eigenvectors:
['0.8165', '-0.0000', '-0.0000', '-0.4082', '-0.0000', '-0.4082', '-0.0000', '-0.0000']

Eigenvalues:
4.0000
Eigenvectors:
['0.5774', '-0.0000', '-0.0000', '0.5774', '-0.0000', '0.5774', '-0.0000', '-0.0000']

Eigenvalues:
-2.0000
Eigenvectors:
['0.4249', '-0.0000', '-0.3486', '-0.2125', '-0.3486', '-0.2125', '-0.0000', '0.6972']

Eigenvalues:
4.0000
Eigenvectors:
['0.2155', '-0.0000', '0.5356', '0.2155', '0.5356', '0.2155', '-0.0000', '0.5356']

Eigenvalues:
-2.0000
Eigenvectors:
['0.2666', '0.0000', '0.7651', '-0.2013', '-0.3277', '-0.0653', '0.0000', '-0.4374']

Eigenvalues:
-2.0000
Eigenvectors:
['-0.1588', '0.0000', '-0.1062', '-0.5324', '0.3667', '0.6912', '0.0000', '-0.2605']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000', '0.0000']
